# Overview / Background for Math Used

Our team's goal with this project is to develop a process for measuring and visualizing the evolving sense of controversial constitutional terminology. The bulk of this work hinges on Word2Vec, an NLP API that allows for the embedding of words as vectors. In vectorizing words, we open up avenues for the analysis of linguistic relationships through geometric means. 

In essence the idea is, given a set of historical documents, we want to generate the "optimal" embedding (which we will discuss more later), and then try to extract a geometric context for controversial words, such as "arms". 

Word embeddings need to capture an incredible amount of complexity and variation, so the word vectors we produce have dimension on the order of tens. In these high-dimensional spaces, the Euclidean norm starts to lose it's ability to "fairly" represent distance, so we instead use the cosine-norm for all distance calculations.

$$ cosine\_norm(u,v) = \frac{u \cdot v}{|u|~|v|} $$ 

Another side-effect of high-dimensions is the obvious difficulty to produce an intelligible visualization. For this reason, in the Network Creation section of our project, we implemented Singular Value Decomposition. SVD is a process for decomposing a rank-r matrix into a sum of r rank-1 matrices. 

Suppose our word embedding process has generated a set of 2000 word vectors of dimension 40 (these are realistic values for this project). These vectors "live" in the standard basis of $R^{40}$. Let's see if we can simplify that basis into something more amenable to visualization while losing as little information as possible. We place all of our vectors into a matrix A as row vectors.
$$ A = \sigma_1 \vec{u_1} \vec{v_1}^T + \sigma_2 \vec{u_2} \vec{v_2}^T ... + \sigma_r \vec{u_r} \vec{v_r}^T$$
It is relevant here because, much like diagonalization, it allows one to obtain a potentially more useful basis for a set of vectors. For our networks, we populate $A$ with our word vectors as rows, and then attain an approximation for A by summing our  three $\sigma_i \vec{u_i} \vec{v_i}^T$ terms with highest corresponding singular values ($\sigma_1,\sigma_2,\sigma_3$). Another way of saying this is (supposing we order our singular values in descending order):

$$A \approx \sigma_1 \vec{u_1} \vec{v_1}^T + \sigma_2 \vec{u_2} \vec{v_2}^T + \sigma_3 \vec{u_3} \vec{v_3}^T$$

This approximation allows us to create a new, 3-dimensional basis for our word vectors:

$$B_{reduced} = \{ \vec{v_1},\vec{v_2},\vec{v_3}\} $$

The basis vectors for $B_{reduced}$ still live in $R^{40}$. However, by projecting each of our word vectors onto this basis, we can achieve an approximation for our word vectors in $R^3$. This lets us visualize the structure of a 40-dimensional space with a graph in 3 dimensions.

# Structure


  * For each era:
    * Parse the relevant set of documents
    * For each "reasonable" set of constructor parameters
      * Initialize a Word2Vec model with parameters
      * Calculate WordSim353 similarity Spearmen correlation for model
    * Choose the model which maximizes WordSim353 Spearmen
    * Analyze the "neighborhood" of chosen words through:
      * Creation of a graph based on Word2Vec's word similarity metric
      %* K-Means Clustering of embedding

# Demonstration

## Parsing

In this step, we take in a set of text documents and tokenize it by sentences and then by words. The result is a list of lists of words, which we will pass to the Word2Vec constructor shortly.

In [1]:
import gensim
from gensim.models import Word2Vec
import sys, re, os
import nltk
from nltk.tokenize import word_tokenize
from eval_embedding import eval_sim
SENTENCE_TOKENIZER = nltk.data.load('./nltk_data/tokenizers/punkt/english.pickle') 
QUOTES = re.compile("\u201c|\u201d")

def read_dir(path):
    """
    @param path: (type = str) path to dir that containes files
    @return: (type=str)
    """
    assert os.path.exists(path)
    file_names = os.listdir(path)
    return read(file_names, path)

def read(file_names, path):
    """
    Reads in a text file as a string, returning the stringified version
    @param file_names: (type=list<str>) files to be read
    @param path: (type = str) path to dir that containes files
    @return: (type=str)
    """
    data = ""
    for file_name in file_names:
        with open(path + "/" + file_name, "r") as f:
            data += re.sub(QUOTES, "\"", f.read())
            data += "\n"
    return data

def parse(text):
    """
    Parses a stringified file into sentences, a list of lists of words
    @param text: (type=str) text to parse
    @return: (type=list<list<str>>) parsed text 
    """
    sentences = SENTENCE_TOKENIZER.tokenize(text)
    sentences = [word_tokenize(sentence) for sentence in sentences]
    return sentences



data = read_dir("Modern_Era/2A")
sentences = parse(data)





C:\Users\Duffy\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ModuleNotFoundError: No module named 'web'

## Embedding/Optimization

We now take the parsed text from the previous step, and pass it to our embedding initializer. We chose 3 model parameters to optimize over:
* size: the dimension of the produced word vectors
* window: the 'radius' around a word that Word2Vec will inspect
* min_count: the minimum frequency for a word to be included in the model

We chose acceptable ranges for each of those parameters and then evaluated every Word2Vec embedding for this particular text within those parameter ranges. We use WordSim353's similarity metric, which measures the correlation of cosine similarity of word vectors with human reported similarity of words. The model that we output is the model with the highest WordSim353 similarity correlation (as similarity is central to our processing of the embedding).

In [2]:
def init_model(sentences):
    """
    Initializes optimal Word2Vec model for text from sentences
    @param sentences: (type=list<list<str>>) a list of lists of words
    @return: (type=Word2Vec) model
    """
    best_spearmen = 0
    best_model = None
    best_params = (0,0,0)
    count = 0
    for dim in range(20, 61):
        for window in range(5, 11):
            for min_count in range(2,5):
                model = Word2Vec(sentences, min_count=min_count, window=window, size=dim)
                similarity_spearmen = eval_sim(model)
                if similarity_spearmen > best_spearmen:
                    best_model = model
                    best_params = (min_count, window, dim)
                    best_spearmen = similarity_spearmen
                #print(str(count) + ": " + str(similarity_spearmen))
                count += 1
    print("min_count: " + str(best_params[0]), 
        "window: " + str(best_params[1]), 
        "dim: " + str(best_params[2]))
    print("correlation: " + str(best_spearmen))
    return best_model

model = init_model(sentences)
#model.save("second_amendment_modern_demo.bin")
model.save("second_amendment.bin")


NameError: name 'sentences' is not defined

## Neighborhood Creation/Dimensionality Reduction

We now use Word2Vec's built in similarity measure to extract the 10 most similar words to our "main_word". This set of 11 words constitutes a "neighborhood". We perform SVD on this neighborhood of word vectors and pass the reduced dimension word vectors along to the visualization stage.

In [12]:
import numpy as np
class Neighborhood:
    def __init__(self, word, model, neighbors=10):
        """
        @param word: (type=str) 
        @param model: (type=Word2Vec Model) <(neighboring_word, edge_weight)>
        @field word: (type=str)
        @field similarity_neighbors: (type=list<tuple<str, float>>) neighbors to word based on 
        @field proximity_neighbors: (type=list<tuple<str, float>>) neighbors to word based on cosine_distance 
        """
        self.word = word
        self.similarity_neighbors = model.wv.most_similar(positive=[word], topn=neighbors)

def get_neighboring_words(word, model, n=10, verbose=False):
    n = Neighborhood(word, model, neighbors=n)
    if verbose:
        print(n.word)
        print("Similarity Neighbors")
    words = []
    for neighbor in n.similarity_neighbors:
        if verbose:
            print(neighbor)
        words.append(neighbor[0])
    return words, n.similarity_neighbors

def get_svd_from_words(model, words, verbose=False):
    vecs = [model.wv[word] for word in words]
    mat = np.stack(vecs, axis=0)
    if verbose:
        print('shape of word embeddngs matrix:', mat.shape) #shape is (neighbors,32), neighbors defaults to 10
    U, s, V = np.linalg.svd(mat)
    if verbose:
        print('singular values:', s)   # Take a quick look at svd_test.py (run it) if you want to convince yourself of how svd works for m by n matrices
               # We basically want the first three row vectors in V; these are the eigenvectors that explain most of the variation in the rows (i.e. word embeddings) of the original matrix.
    return U, s, V

def get_coords_from_svd_projection(V, model, words, verbose=False):
    V_cut = V[:3,:]
    if verbose:
        print('matrix after removing less import eigenvectors:')
        print(V_cut)
        print('squared row magnitudes:')
        for i in range(3):     # this yields 1.0 every time, i.e. to compute projection coordinates we can ignore the a.a on the denominator (V_cut has unitary rows)
            print(V_cut[i,:].dot(V_cut[i,:]))


    #for each word, associate it with its projection coordinates in the V_cut basis.
    # This associates each words with a ordered triple of points, which allows us to graph in 3d,
    # or 2d (you can just use the first two coordinates if you want). One idea for the future would
    #be to label the axes of the graph with the word that the corresponding basis vector of our graph is closest to.
    return V_cut, {w: V_cut.dot(model.wv[w]) for w in words}

"""
Main utility method for any users of this file. Bundles up three sub-processes to allow you 
to get a three dimensional space on which to plot the similar words to the given one based on some model

The current method computes the svd of the similar words, along with the original word, but throws out words of length
2 or smaller by default.
"""
def get_points_from_word_and_model(word, model_path, verbose=False, bigger_than=2):
    model = Word2Vec.load(model_path)
    
    #gets the n "most similar" words to the initial word in this model. 
    #Also returns a dict containing those similarity scores, which is not used in this method
    words,_ = get_neighboring_words(word,model, n=10, verbose=verbose)

    # an intermediate processing step here that removes small words and adds in the main word we're considering?
    cond = (lambda w: len(w) > bigger_than) if bigger_than > 0 else (lambda w: True)
    words = [word] + [w for w in words if cond(w)]

    # computes the svd of the matrix containing the embeddings of the 10 most similar words as rows. 
    # U and s are not used, but are left in for readability. s contains the singular values, set verbose=True to print them.
    U, s, V = get_svd_from_words(model, words, verbose=verbose)
    
    basis, coords = get_coords_from_svd_projection(V, model, words, verbose=verbose)

    return basis, coords, model

word = "arms"
model_path = "second_amendment.bin"
basis, coords, _ = get_points_from_word_and_model(word, model_path)


## Neighborhood Visualization
In this stage, we plot the neighborhood in our new 3-dimensional basis and create a labeled visualization of it.

In [15]:
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from nltk.cluster.util import cosine_distance
#working example
"""
from plotly.graph_objs import Scatter, Layout

#when using jupyter notebooks, uncomment the following line and change function call below to plotly.offline.iplot
#plotly.offline.init_notebook_mode(connected=True)

plotly.offline.plot({
    "data": [Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": Layout(title="hello world")
})
"""
plotly.offline.init_notebook_mode(connected=True)

print(basis.shape[1])
print(model.wv)
#coords should be a dictionary from a word to a 3d position
def generate_network(main_word, coords):

    #next step: generate edges.
    #   In general, we can modify or optionize this however we want to get whatever edge structures we find interesting. 
    #   --We might, for example, create more Neighborhoods (one for each "similar" word) to see if we get any connections within the 1-level-away words.
    #   --We could also try projecting some words from these new neighborhoods (maybe the top three from each?) onto our basis (just dot product) and getting those edges as well.
    e_dict = {}
    for i in range(3):
        e_dict[coord_map[i] + '_e'] = sum([[main_coords[i], coords[w][i], None] for w in id_to_name if w != main_word], [])
        e_dict[coord_map[i] + '_n'] = [coords[w][i] for w in id_to_name]

    return e_dict

def plot_network(g_dict, axis_titles):
    trace1=Scatter3d(
        x=g_dict['x_e'], y=g_dict['y_e'], z=g_dict['z_e'], mode='lines', line=Line(
                    color='rgb(125,125,125)', width=1), hoverinfo='none')

    trace2=Scatter3d(x=g_dict['x_n'], y=g_dict['y_n'], z=g_dict['z_n'], mode='markers', marker=Marker(
        symbol='dot', size=6, color='rgb(175,175,175)', line = Line(
            color='rgb(50,50,50)', width=0.5)), text=id_to_name, hoverinfo='text')

    #print(axis_titles)
    axes = [dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=' ') for i in range(3)]
 
    title = 'word relationships with respect to ' + main_word
    layout=Layout(title=title,width=1000, height=1000, showlegend=False, scene=Scene(
        xaxis=XAxis(axes[0]), yaxis=YAxis(axes[1]), zaxis=ZAxis(axes[2])), margin=Margin(t=100), hovermode='closest')
    
    data = Data([trace1, trace2])
    fig=Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='arms_network.html')
    
def get_closest_basis_words(basis, model):
    maps = {}
    for i in range(basis.shape[0]):
        curr_bas = basis[i,:]
        closest = ('', 10)
        for key in model.wv.vocab:
#            if len(key) < 3:
#                continue
            word = key
            vec = model.wv[key]
            dist = abs(cosine_distance(vec, curr_bas))
            if dist < closest[1]:
                closest = (word, dist)
        maps[i] = closest
    return maps
                


#basis isn't going to be used yet, but we might label the axes in the future
#generate node-id mappings
main_word = word
name_to_id = {}
id_to_name = []
temp_id = 0
for w in coords:
    name_to_id[w] = temp_id
    temp_id +=1
    id_to_name.append(w)
main_coords = coords[main_word] 
coord_map= ['x','y','z']

#generate network
g_dict = generate_network(main_word, coords)

#plot network
plot_network(g_dict, get_closest_basis_words(basis, model))


40
